# Documentation: Topology

The topology of all elements implemented on NeoPZ are grouped in pztopology. The classes associated to this namespace are listed in the following:

 
1. TPZPoint
2. TPZLine
3. TPZQuadrilateral
4. TPZTriangle
5. TPZCube
6. TPZPrism
7. TPZPyramid
8. TPZTetrahedron

## Main atributions of the class Topology

a. Definition of number of coners, sides, dimension and faces.

In [ ]:
pztopology::TPZQuadrilateral::NCornerNodes

b. 